<a href="https://colab.research.google.com/github/jaehyun-shin/lanchain_tutorials/blob/main/2_ServingWithLangserve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 설치 및 환경 세팅

LangServe는 개발자가 LangChain 체인을 REST API로 배포할 수 있도록 지원합니다. LangChain을 사용하기 위해 LangServe를 사용할 필요는 없지만 이 가이드에서는 LangServe를 사용하여 앱을 배포하는 방법을 보여줍니다.

해당 환경은 Jupyter Notebook이 아닌 Python 파일을 만든 다음에 실행합니다.

In [2]:
pip install "langserve[all]" langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = userdata.get("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
LLM_MODEL = "gpt-3.5-turbo"

# Server 구현

응용 프로그램에 대한 서버를 만들기 위해 파일을 만듭니다. 여기에는 응용 프로그램을 제공하기 위한 논리가 포함됩니다. 다음 세 가지로 구성됩니다`.serve.py`

1. 위에서 방금 구축한 체인의 정의
2. FastAPI 앱
3. 체인을 서비스할 경로의 정의로, 다음과 같이 수행됩니다. `langserve.add_routes`

In [ ]:
import os
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langserve import add_routes

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Create model
model = ChatOpenAI()

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

코드를 실행시킵니다

`python serve.py`

해당 체인이 http://localhost:8000에서 제공됩니다.

langserve는 http://localhost:8000/chain/playground/ 를 통해 내장 UI가 함께 제공됩니다.

# 클라이언트

이제 서버와 상호작용을 하기 위해 클라이언트에서 실행해보겠습니다.

In [4]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "korean", "text": "hi, this is langchain tutorails"})

ConnectError: [Errno 99] Cannot assign requested address